ENEM Study - How is education performing in Brazil?
===========================================

Overview
-------------
This is a report that compares ENEM (Exame Nacional do Ensino Médio - High School National Exam) with other metrics in order to analyse:

* statistical accuracy
* improvement over years
* comparison among different regions of the country

The project consists of different steps of data manipulation and is a good Data Science exercise.

Data Source
-----------------

We will be gathering our data from http://inep.gov.br/microdados , where they are available for download on the form of a zipped csv file. As you might have already noticed, each file has around 3GB of size, so we will have to use a Database to perform our queries and extract data of interest. To keep it simple, we will be using PostGRESQL.

In [4]:
import db
import sqlalchemy

session = db.get_db_session(create_schema=False)

The block below will only be run once to import data in the database. Please be careful!

In [14]:
import os
import os.path
import csv
session = db.get_db_session(create_schema=True)
csv_path = 'D:/Projects/Data/ENEM/microdados_enem2017/DADOS/MICRODADOS_ENEM_2017.csv'
with open(csv_path,'r') as f:
    reader = csv.reader(f)
    header = next(reader)[0].split(';')
    counter = 0
    for row in reader:
        values = row[0].split(';')
        items = dict(zip(header,values))
        
        for key in items.keys():
            if items[key]=='':
                items[key]=None
                
        #checks if the person finished the exam, those who did not finish will be discarded
        if items['NU_NOTA_CN'] and items['NU_NOTA_CH'] and items['NU_NOTA_LC'] and items['NU_NOTA_MT'] and items['NU_NOTA_REDACAO']:
            exame = db.Exame()
            exame.candidato_id = int(items['NU_INSCRICAO']) 
            exame.ano = int(items['NU_ANO']) if items['NU_ANO'] else None
            exame.idade = int(items['NU_IDADE']) if items['NU_IDADE'] else None
            exame.racial_id = int(items['TP_COR_RACA']) if items['TP_COR_RACA'] else None
            exame.sexo = items['TP_SEXO']
            exame.casado_id = int(items['TP_ESTADO_CIVIL']) if items['TP_ESTADO_CIVIL'] else None
            exame.nacional_id = int(items['TP_NACIONALIDADE']) if items['TP_NACIONALIDADE'] else None
            exame.ensinomedio_id = int(items['TP_ST_CONCLUSAO']) if items['TP_ST_CONCLUSAO'] else None
            exame.tipoensinomedio_id = int(items['TP_ESCOLA']) if items['TP_ESCOLA'] else None
            if items['IN_TREINEIRO']=='1':
                exame.treineiro = True 
            elif items['IN_TREINEIRO']=='0':
                exame.treineiro = False
            else:
                exame.treineiro = None
            exame.nota_cn = float(items['NU_NOTA_CN'])
            exame.nota_ch = float(items['NU_NOTA_CH'])
            exame.nota_lc = float(items['NU_NOTA_LC'])
            exame.nota_mt = float(items['NU_NOTA_MT']) 
            exame.nota_red = float(items['NU_NOTA_REDACAO']) 


            if items['CO_MUNICIPIO_PROVA']:
                if not session.query(db.Local).filter_by(id=int(items['CO_MUNICIPIO_PROVA'])).all():
                    local = db.Local()
                    local.id = int(items['CO_MUNICIPIO_PROVA'])
                    local.municipio = items['NO_MUNICIPIO_PROVA']
                    local.estado = items['SG_UF_PROVA']
                    session.add(local)
                    session.commit()
                exame.exame_local_id = int(items['CO_MUNICIPIO_PROVA']) 


            if items['CO_MUNICIPIO_NASCIMENTO']:
                if not session.query(db.Local).filter_by(id=int(items['CO_MUNICIPIO_NASCIMENTO'])).all():
                    local = db.Local()
                    local.id = int(items['CO_MUNICIPIO_NASCIMENTO'])
                    local.municipio = items['NO_MUNICIPIO_NASCIMENTO']
                    local.estado = items['SG_UF_NASCIMENTO']
                    session.add(local)
                    session.commit()
                exame.local_nasc_id = int(items['CO_MUNICIPIO_NASCIMENTO'])

            if items['CO_MUNICIPIO_RESIDENCIA']:
                if not session.query(db.Local).filter_by(id=int(items['CO_MUNICIPIO_RESIDENCIA'])).all():
                    local = db.Local()
                    local.id = int(items['CO_MUNICIPIO_RESIDENCIA'])
                    local.municipio = items['NO_MUNICIPIO_RESIDENCIA']
                    local.estado = items['SG_UF_RESIDENCIA']
                    session.add(local)
                    session.commit()
            exame.residencia_id = int(items['CO_MUNICIPIO_RESIDENCIA'])

            session.add(exame)
            counter += 1
            
            if counter%20000 == 0:
                session.commit()
                print('Inserted: '+str(counter))

        session.commit()    #commits to session
        session.close()     #closes session


Inserted: 20000
Inserted: 40000
Inserted: 60000
Inserted: 80000
Inserted: 100000
Inserted: 120000
Inserted: 140000
Inserted: 160000
Inserted: 180000
Inserted: 200000
Inserted: 220000
Inserted: 240000
Inserted: 260000
Inserted: 280000
Inserted: 300000
Inserted: 320000
Inserted: 340000
Inserted: 360000
Inserted: 380000
Inserted: 400000
Inserted: 420000
Inserted: 440000
Inserted: 460000
Inserted: 480000
Inserted: 500000
Inserted: 520000
Inserted: 540000
Inserted: 560000
Inserted: 580000
Inserted: 600000
Inserted: 620000
Inserted: 640000
Inserted: 660000
Inserted: 680000
Inserted: 700000
Inserted: 720000
Inserted: 740000
Inserted: 760000
Inserted: 780000
Inserted: 800000
Inserted: 820000
Inserted: 840000
Inserted: 860000
Inserted: 880000
Inserted: 900000
Inserted: 920000
Inserted: 940000
Inserted: 960000
Inserted: 980000
Inserted: 1000000
Inserted: 1020000
Inserted: 1040000
Inserted: 1060000
Inserted: 1080000
Inserted: 1100000
Inserted: 1120000
Inserted: 1140000
Inserted: 1160000
Inserted:

As you may have noticed, we are not adding people who did not have score on every test (math, science, writting, humanities and communication) because this exercise consists of only people who have actually finished the test, or in other words, who did not give up in the middle. For the 2017 edition, approx. 4.5 million applicants did their tests to the end, which already represents a considerable number of data to handle. See how important it was to use a database?

First we onsult how many entries are there.

In [16]:
session = db.get_db_session(create_schema=False)
print(session.query(db.Exame).filter(db.Exame.idade<19,db.Exame.treineiro == False).count())
session.close()

1504360


Now let´s start filtering the top 1000 scores in Math.

In [22]:
import pandas as pd
import numpy as np
from sqlalchemy.orm import joinedload
from sqlalchemy.orm import aliased

session = db.get_db_session(create_schema=False)
nasc = aliased(db.Local)
exame = aliased(db.Local)
res = aliased(db.Local)
'''
df = pd.read_sql(session.query(db.Exame.sexo,
                               db.Exame.nota_mt,
                               db.Exame.nota_ch,
                               db.Exame.nota_red,
                               db.Exame.nota_lc,
                               db.Exame.nota_cn,
                               db.Exame.idade,
                               exame.municipio.label('municipio_exame'),
                               exame.estado.label('estado_exame'),
                               nasc.municipio.label('municipio_nasc'),
                               nasc.estado.label('estado_nasc'),
                               res.municipio.label('municipio_res'),
                               res.estado.label('estado_res')
                              ).
                 order_by(db.Exame.nota_mt.desc()).limit(10).statement,session.bind) 
                 
'''

df = pd.read_sql(session.query(db.Exame).with_labels().filter(db.Exame.idade<19,db.Exame.treineiro==False).
                 options(joinedload(db.Exame.exame_local),joinedload(db.Exame.residencia),joinedload(db.Exame.local_nasc),
                        joinedload(db.Exame.racial),joinedload(db.Exame.nacional),joinedload(db.Exame.casado)).
                 order_by(db.Exame.nota_mt.desc()).
                 limit(15000).
                 statement,session.bind)
#cleans ids
df = pd.concat([df['exame_idade'],
                df['nacionalidade_1_nacionalidade'],
               df['estadocivil_1_estado_civil'],
               df['exame_sexo'],
               df['exame_nota_cn'],
               df['exame_nota_ch'],
               df['exame_nota_lc'],
               df['exame_nota_mt'],
               df['exame_nota_red'],
               df['racial_1_racial_id'],
               df['local_1_municipio'],
               df['local_1_estado'],
               df['local_2_municipio'],
               df['local_2_estado'],
               df['local_3_municipio'],
               df['local_3_estado']],axis=1,
               keys=['idade','nacionalidade','casado','sexo','nota_cn','nota_ch','nota_lc','nota_mt',
                                  'nota_red','raca','mun_exame','estado_exame','mun_res','estado_res','mun_nasc','estado_nasc'])
session.close()



Now we see the 10 first rows to make sure we are ok.

In [23]:
df.head(10)

,idade,nacionalidade,casado,sexo,nota_cn,nota_ch,nota_lc,nota_mt,nota_red,raca,mun_exame,estado_exame,mun_res,estado_res,mun_nasc,estado_nasc
0,18,Brasileiro(a),Solteiro(a),M,585.4,673.2,581.3,993.9,600.0,Branca,Maceió,AL,Maceió,AL,Maceió,AL
1,17,Brasileiro(a),Solteiro(a),M,812.1,813.1,736.6,993.9,820.0,Branca,Osasco,SP,São Paulo,SP,Osasco,SP
2,18,Brasileiro(a),Solteiro(a),M,833.9,781.0,666.6,993.9,840.0,Branca,Belo Horizonte,MG,Belo Horizonte,MG,Belo Horizonte,MG
3,17,Brasileiro(a),Solteiro(a),M,813.6,808.4,695.6,993.9,900.0,Parda,Recife,PE,Recife,PE,Recife,PE
4,16,Brasileiro(a),Solteiro(a),M,771.5,705.6,607.9,992.8,840.0,Parda,Sete Lagoas,MG,Sete Lagoas,MG,Sete Lagoas,MG
5,17,Brasileiro(a),Solteiro(a),M,736.2,718.9,638.4,992.8,860.0,Branca,Araras,SP,Araras,SP,Araras,SP
6,17,Brasileiro(a),Solteiro(a),M,739.7,765.8,635.3,990.2,760.0,Branca,Aracaju,SE,Aracaju,SE,Aracaju,SE
7,18,Brasileiro(a),Solteiro(a),M,705.9,668.6,627.6,990.2,680.0,Parda,Rio de Janeiro,RJ,Rio de Janeiro,RJ,Rio de Janeiro,RJ
8,17,Brasileiro(a),Solteiro(a),M,721.2,681.8,600.7,989.7,800.0,Branca,Recife,PE,Recife,PE,Recife,PE
9,18,Brasileiro(a),Solteiro(a),M,731.7,734.0,672.9,989.7,940.0,Branca,Goiânia,GO,Rio Verde,GO,Goiânia,GO


In [24]:
df.groupby('estado_res').count().sort_values(['nota_mt'],ascending=False)

,idade,nacionalidade,casado,sexo,nota_cn,nota_ch,nota_lc,nota_mt,nota_red,raca,mun_exame,estado_exame,mun_res,mun_nasc,estado_nasc
estado_res,,,,,,,,,,,,,,,
SP,3805,3805,3745,3805,3805,3805,3805,3805,3805,3805,3805,3805,3805,3805,3805
MG,2299,2299,2261,2299,2299,2299,2299,2299,2299,2299,2299,2299,2299,2299,2299
RJ,1532,1532,1496,1532,1532,1532,1532,1532,1532,1532,1532,1532,1532,1532,1532
CE,909,909,897,909,909,909,909,909,909,909,909,909,909,909,909
PR,824,824,802,824,824,824,824,824,824,824,824,824,824,824,824
PE,644,644,629,644,644,644,644,644,644,644,644,644,644,644,644
RS,604,604,595,604,604,604,604,604,604,604,604,604,604,604,604
GO,548,548,534,548,548,548,548,548,548,548,548,548,548,548,548
BA,495,495,481,495,495,495,495,495,495,495,495,495,495,495,495


In [45]:
df[df['sexo']=='F'].count()

idade            768316
nacionalidade    768316
casado           757356
sexo             768316
nota_cn          768316
nota_ch          768316
nota_lc          768316
nota_mt          768316
nota_red         768316
raca             768316
mun_exame        768316
estado_exame     768316
mun_res          738158
estado_res       738158
mun_nasc         768316
estado_nasc      768316
dtype: int64

In [37]:
from sqlalchemy import func

session = db.get_db_session(create_schema=False)
df = pd.read_sql(session.query(db.Exame).with_labels().filter(
                                (db.Exame.nota_ch+db.Exame.nota_red+db.Exame.nota_lc+db.Exame.nota_cn+db.Exame.nota_mt)/5>=450,
                                db.Exame.treineiro==False,
                                db.Exame.idade<19).
                 options(joinedload(db.Exame.exame_local),joinedload(db.Exame.residencia),joinedload(db.Exame.local_nasc),
                        joinedload(db.Exame.racial),joinedload(db.Exame.nacional),joinedload(db.Exame.casado)).
                 statement,session.bind)
session.close()

df = pd.concat([df['exame_idade'],
                df['nacionalidade_1_nacionalidade'],
               df['estadocivil_1_estado_civil'],
               df['exame_sexo'],
               df['exame_nota_cn'],
               df['exame_nota_ch'],
               df['exame_nota_lc'],
               df['exame_nota_mt'],
               df['exame_nota_red'],
               df['racial_1_racial_id'],
               df['local_1_municipio'],
               df['local_1_estado'],
               df['local_2_municipio'],
               df['local_2_estado'],
               df['local_3_municipio'],
               df['local_3_estado']],axis=1,
               keys=['idade','nacionalidade','casado','sexo','nota_cn','nota_ch','nota_lc','nota_mt',
                                  'nota_red','raca','mun_exame','estado_exame','mun_res','estado_res','mun_nasc','estado_nasc'])


In [38]:
group_by_sexo = df.groupby('sexo').count()
group_by_sexo.loc["F"]['idade']/(group_by_sexo.loc["F"]['idade'] + group_by_sexo.loc["M"]['idade'])

0.596935747028203

In [39]:
print(df.shape)

(1287100, 16)


In [44]:
df = df.sort_values('nota_mt',ascending=True)
df.head(30)

,idade,nacionalidade,casado,sexo,nota_cn,nota_ch,nota_lc,nota_mt,nota_red,raca,mun_exame,estado_exame,mun_res,estado_res,mun_nasc,estado_nasc
1287099,17,Brasileiro(a),Solteiro(a),M,562.5,592.1,521.0,0.0,820.0,Parda,Governador Valadares,MG,Mendes Pimentel,MG,Governador Valadares,MG
1287088,18,Brasileiro(a),Solteiro(a),F,502.2,581.8,510.5,0.0,700.0,Branca,Barbacena,MG,Barbacena,MG,Barbacena,MG
1287089,18,Brasileiro(a),None,F,376.0,580.5,598.9,0.0,780.0,Branca,Jaraguá do Sul,SC,Jaraguá do Sul,SC,Jaraguá do Sul,SC
1287090,17,Brasileiro(a),Solteiro(a),F,492.8,567.9,484.1,0.0,840.0,Parda,Brasília,DF,Brasília,DF,Brasília,DF
1287091,18,Brasileiro(a),Solteiro(a),M,584.0,592.5,589.9,0.0,540.0,Parda,Bauru,SP,Monte Carmelo,MG,Bauru,SP
1287092,17,Brasileiro(a),Solteiro(a),M,475.5,645.5,490.1,0.0,700.0,Branca,Cacoal,RO,Cacoal,RO,Cacoal,RO
1287098,17,Brasileiro(a),Solteiro(a),F,493.0,638.4,556.6,0.0,680.0,Branca,Santana de Parnaíba,SP,Santo André,SP,Santana de Parnaíba,SP
1287094,17,Brasileiro(a),Solteiro(a),M,555.3,595.7,587.4,0.0,740.0,Branca,Paulista,PE,Olinda,PE,Paulista,PE
1287095,17,Brasileiro(a),Solteiro(a),M,423.2,600.3,576.4,0.0,720.0,Branca,Mucuri,BA,Palmas,TO,Mucuri,BA
1287096,18,Brasileiro(a),Solteiro(a),F,540.6,660.9,585.1,0.0,800.0,Parda,Várzea Grande,MT,Cuiabá,MT,Várzea Grande,MT
